In [ ]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
from collections import Counter
import string
import re
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

In [ ]:
train_file_path = 'train.csv'
test_file_path = 'test.csv'

# Read the CSV file into a DataFrame.
# If the file does not have headers, specify header=None and assign a column name.
df = pd.read_csv(train_file_path, header=None, names=['sentence'])
df_test = pd.read_csv(test_file_path, header=None, names=['sentence'])

#### EDA on train dataset

In [ ]:
# DataFrame structure and general info
print("Training DataFrame Info:")
df.info()

# Display summary statistics
print("\nDataFrame Summary Statistics:")
print(df.describe(include='all'))

# Check the first few rows of the DataFrame
print("\nFirst Few Rows:")
print(df.head())

#### EDA on test dataset

In [ ]:
# DataFrame structure and general info
print("Testing DataFrame Info:")
df_test.info()

# Display summary statistics
print("\nDataFrame Summary Statistics:")
print(df_test.describe(include='all'))

# Check the first few rows of the DataFrame
print("\nFirst Few Rows:")
print(df_test.head())

#### EDA: Comparision between train and test dataset

In [ ]:
# Drop Duplicates and Compare
print("\nLength of Train Dataset before dropping duplicates:", len(df))
print("Length of Test Dataset before dropping duplicates:", len(df_test))

df = df.drop_duplicates(subset=['sentence'])
df_test = df_test.drop_duplicates(subset=['sentence'])

print("\nLength of Train Dataset after dropping duplicates:", len(df))
print("Length of Test Dataset after dropping duplicates:", len(df_test))

unique_to_df1 = df[~df['sentence'].isin(df_test['sentence'])]
unique_to_df2 = df_test[~df_test['sentence'].isin(df['sentence'])]
common_rows = pd.merge(df, df_test, on='sentence')

# Create counts for visualization
counts = {
    'Unique to Train': len(unique_to_df1),
    'Unique to Test': len(unique_to_df2),
    'Common Rows': len(common_rows)
}

print("\nUnique to Train:", len(unique_to_df1))
print("Unique to Test:", len(unique_to_df2))
print("Common rows:", len(common_rows))

# Plotting comparison for counts
plt.bar(counts.keys(), counts.values(), color=['skyblue', 'salmon', 'lightgreen'])
plt.title("Comparison of Rows Between Test and Train DataFrames (Counts)")
plt.ylabel("Number of Rows")
plt.show()

# Calculate percentages for better insight
total_rows = len(df) + len(df_test)
percentages = {key: (value / total_rows) * 100 for key, value in counts.items()}

print("\nPercentage of Unique to Train: {:.2f}%".format(percentages['Unique to Train']))
print("Percentage of Unique to Test: {:.2f}%".format(percentages['Unique to Test']))
print("Percentage of Common Rows: {:.2f}%".format(percentages['Common Rows']))

# Plotting percentage comparison
plt.bar(percentages.keys(), percentages.values(), color=['skyblue', 'salmon', 'lightgreen'])
plt.title("Percentage Comparison Between Test and Train DataFrames")
plt.ylabel("Percentage")
plt.show()

# Assertions for sanity checks
assert len(df) == len(unique_to_df1) + len(common_rows), "Mismatch in Train dataset counts!"
print("Train dataset assertion passed ✅")

assert len(df_test) == len(unique_to_df2) + len(common_rows), "Mismatch in Test dataset counts!"
print("Test dataset assertion passed ✅")

assert sum(counts.values()) == len(df) + len(df_test) - len(common_rows), "Total row counts assertion passed ✅"
print("Total row counts assertion passed ✅")



#### Sentence Parsing
Converting the given sentence string to a list of tuples.

In [ ]:
df['parsed_sentence'] = df['sentence'].apply(lambda x: ast.literal_eval(x))
df_test['parsed_sentence'] = df_test['sentence'].apply(lambda x: ast.literal_eval(x))

#### EDA on train dataset

In [ ]:
num_sentences = len(df)
print("\nTotal number of sentences:", num_sentences)
# Display DataFrame information
print("\nDataFrame Info:")
print(df.info())

In [ ]:
# print the first row with all columns
print("After converting from string to list of tuples, the parsed sentence in first row:")
print(df['parsed_sentence'].iloc[0])
print(type(df['parsed_sentence'].iloc[0]))
# print more info about the column parsed. like, the datatype and stuff
# print(df['parsed'].apply(type).value_counts())
print("\nFirst value in first row of the parsed column:")
print(df['parsed_sentence'].iloc[0][0])
print(type(df['parsed_sentence'].iloc[0][0]))

In [ ]:
# Calculate sentence lengths (number of word-tag pairs per sentence)
df['sentence_length'] = df['parsed_sentence'].apply(lambda sentence: len(sentence))
print("\nSentence length statistics:")
print(df['sentence_length'].describe())

print("\nLength of sentences for first few rows:")
print(df['sentence_length'].head())

In [ ]:
# Visualize the distribution of sentence lengths
plt.figure(figsize=(8, 6))
df['sentence_length'].hist(bins=20, color='skyblue', edgecolor='black')
plt.title("Sentence Length Distribution")
plt.xlabel("Number of Words per Sentence")
plt.ylabel("Frequency")
plt.grid(False)
plt.show()

#### Separating word and corresponding tag pair for each word in all sentences 

In [ ]:
# Create a list of dictionaries where each dictionary represents a word-tag pair.
records = []
for sentence in df['parsed_sentence']:
    for word, tag in sentence:
        records.append({'word': word, 'tag': tag})

# Create a new DataFrame from the records.
df_words = pd.DataFrame(records)
print(df_words)

#### EDA: word-tag distribution

In [ ]:
# Check basic information and types
print("Dataset Info:")
df_words.info()

# Summary statistics
print("\nDataset Summary Statistics:")
print(df_words.describe(include='all'))

# Check for missing values
print("\nMissing Values:")
print(df_words.isnull().sum())


In [ ]:
# Count the frequency of each tag
tag_distribution = df_words['tag'].value_counts()
print("Length of tag distribution:", len(tag_distribution))
print("\nTag Frequency Distribution:")
print(tag_distribution)

# Plot the distribution of tags
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.barplot(x=tag_distribution.index, y=tag_distribution.values)
plt.title("Distribution of Tags")
plt.xlabel("POS Tags")
plt.ylabel("Frequency")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Count the frequency of words
word_frequency = df_words['word'].value_counts()
print("\nTop 10 Most Frequent Words:")
print(word_frequency.head(10))

# Plot the top 10 most frequent words
plt.figure(figsize=(10, 6))
sns.barplot(x=word_frequency.head(10).index, y=word_frequency.head(10).values)
plt.title("Top 10 Most Frequent Words")
plt.xlabel("Words")
plt.ylabel("Frequency")
plt.xticks(rotation=45)
plt.show()


### Step-2. Pre-Processing

#### Text Cleaning
- Converting to Lowercase
- Removing URLs if any using regex library
- Removing non-word and non-whitespace characters: punctuations marks, symbols, other special characters

In [ ]:
def clean_word(word):
    # Convert to lowercase
    word = word.lower()
    
    # Remove URLs
    word = re.sub(r'http\S+|www\S+|https\S+', '', word)
    
    # Remove non-word and non-whitespace characters, but not tags like '.'
    word = re.sub(r'[^\w\s]', '', word)
    
    # # Remove alphanumeric words containing digits but leave words like underscores
    # word = re.sub('\b\w*\d\w*\b', '', word)
    
    # Remove words that consist only of underscores
    if re.match(r'^\_+$', word):
        return None  # Remove this word if it consists of only underscores

    # Return cleaned word if it's not purely numeric and not empty
    return word if word else None

def clean_sentence(sentence):
    cleaned_sentence = []
    for word, tag in sentence:
        # Only clean the word, keep the tag unchanged
        cleaned_word = clean_word(word)
        if cleaned_word and tag != 'X' and tag != 'NUM' and tag != '_' and tag != 'SYM':  # Keep valid cleaned words and remove 'X' tags
            cleaned_sentence.append((cleaned_word, tag))
    return cleaned_sentence

# Apply the updated cleaning function to the parsed sentences
df['parsed_sentence'] = df['parsed_sentence'].apply(clean_sentence)
# Doing it on test dataset also
df_test['parsed_sentence'] = df_test['parsed_sentence'].apply(clean_sentence)


In [ ]:
# Extract all words and tags from the parsed sentences
all_words = [word for sentence in df['parsed_sentence'] for word, _ in sentence]
all_tags = [tag for sentence in df['parsed_sentence'] for __, tag in sentence]
print("Total words: ", len(all_words), "Total tags: ", len(all_tags))
# Get unique words and tags
unique_words = set(all_words)
unique_tags = set(all_tags)

print(f"Total Unique Words: {len(unique_words)}")
print(f"Total Unique Tags: {len(unique_tags)}")
print("Few Unique Words:", list(unique_words)[:10])
print("\nFew Unique Tags:", list(unique_tags))


In [ ]:
# Create dictionaries mapping words and tags to indices
word_to_index = {word: idx for idx, word in enumerate(sorted(unique_words))}
tag_to_index = {tag: idx for idx, tag in enumerate(sorted(unique_tags))}

print("\nSample Word to Index Mapping:", list(word_to_index.items())[:10])
print("\nSample Tag to Index Mapping:", list(tag_to_index.items()))

#### Stop word removal

In [ ]:
# # Download stopwords if not already downloaded
# nltk.download('stopwords')

# # Define the stop words set
# stop_words = set(stopwords.words('english'))

In [ ]:
# # Count the total stop words before removal
# total_stop_words_before = sum(
#     word.lower() in stop_words for sentence in df['parsed_sentence'] for word, _ in sentence
# )

# print(f"Total number of stop words before removal: {total_stop_words_before}")

In [ ]:
# def remove_stop_words(sentence):
#     # Remove stop words from the sentence while keeping tags intact
#     return [(word, tag) for word, tag in sentence if word.lower() not in stop_words]

# # Apply the stop word removal function
# df['parsed_sentence'] = df['parsed_sentence'].apply(remove_stop_words)
# df_test['parsed_sentence'] = df_test['parsed_sentence'].apply(remove_stop_words)

In [ ]:
# # Count the total stop words after removal
# total_stop_words_after = sum(
#     word.lower() in stop_words for sentence in df['parsed_sentence'] for word, _ in sentence
# )

# print(f"Total number of stop words after removal: {total_stop_words_after}")


#### Generating Vocabulary

In [ ]:
from collections import defaultdict

# Extract unique tags and words from the parsed sentences to account all the unique tags and words
tags = set()
words = set()

for sentence in df['parsed_sentence']:
    for item in sentence:
        word, tag = item
        tags.add(tag)
        words.add(word)

tags = list(tags)
words = list(words)

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Download necessary NLTK data
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")
nltk.download("omw-1.4")

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to convert NLTK POS tag to WordNet POS tag
def get_wordnet_pos(tag):
    if tag.startswith('V'):  # Verb
        return wordnet.VERB
    elif tag.startswith('N'):  # Noun
        return wordnet.NOUN
    return None  # Ignore other parts of speech

# Function to lemmatize words in a sentence based on their POS tags
def lemmatize_sentence(sentence):
    lemmatized_sentence = []
    for word, tag in sentence:
        wordnet_pos = get_wordnet_pos(tag)
        if wordnet_pos:
            lemmatized_word = lemmatizer.lemmatize(word, wordnet_pos)
            lemmatized_sentence.append((lemmatized_word, tag))
        else:
            lemmatized_sentence.append((word, tag))
    return lemmatized_sentence

# Apply lemmatization to the parsed sentences in the train and test datasets
df['parsed_sentence'] = df['parsed_sentence'].apply(lemmatize_sentence)
df_test['parsed_sentence'] = df_test['parsed_sentence'].apply(lemmatize_sentence)

# Display the first few rows to verify the changes
print("First few rows of lemmatized train dataset:")
print(df['parsed_sentence'].head())

print("\nFirst few rows of lemmatized test dataset:")
print(df_test['parsed_sentence'].head())

#### Transition Probabilities
  
<ul>
  <li> Compute <i>P</i>(<i>tag</i><sub>t+1</sub> | <i>tag</i><sub>t</sub>): 
      the probability of transitioning from one tag to another.
  </li>
  <li> Formula:
      <i>P</i>(<i>tag</i><sub>t+1</sub> | <i>tag</i><sub>t</sub>) =
        <span >
            Count(<i>tag</i><sub>t+1</sub> given <i>tag</i><sub>t</sub>)
        </span>/
        Count(<i>tag</i><sub>t</sub>)
  </li>
</ul>

In [ ]:
# Compute transition probabilities P(tag2 | tag1)
transition_counts = defaultdict(lambda: defaultdict(int))
tag_counts = defaultdict(int)

for sentence in df['parsed_sentence']:
    prev_tag = None
    for _, tag in sentence:
        tag_counts[tag] += 1
        if prev_tag is not None:
            transition_counts[prev_tag][tag] += 1
        prev_tag = tag

# doing laplace smoothing to avoid zero probabilities and adding 1 to numerator and len(tags) to denominator for fair probability distribution 
transition_probs = {
    tag1: {tag2: (transition_counts[tag1][tag2] + 1) / (tag_counts[tag1] + len(tags))
           for tag2 in tags}
    for tag1 in tags
}

print("\nTransition Probabilities:\n", transition_probs)

<p style="font-size:16px">
Note:- Why Use Laplacian Smoothing in HMM?
<ul style="font-size:15px">
    <li> Avoids zero probabilities, preventing the Viterbi algorithm from breaking.
    <li> Encourages generalization, allowing the model to predict unseen sequences.
    <li> Prevents overfitting, distributing probability mass more fairly among possible transitions.
    <li> Handles sparse data, which is common in NLP and speech recognition.
</ul>
</p>

#### Emission Probabilities
  
<ul>
  <li> Compute P ( word | tag ) : 
      <i>the probability of a word being generated by a tag.</i>
  </li>
  <li> Formula:
      <i>P ( word | tag ) = Count(word,tag) / Count(tag)</i>
  </li>
</ul>

In [ ]:
# Compute emission probabilities P(word | tag)
emission_counts = defaultdict(lambda: defaultdict(int))

for sentence in df['parsed_sentence']:
    for word, tag in sentence:
        emission_counts[tag][word] += 1

emission_probs = {
    tag: {word: (emission_counts[tag][word] + 1) / (tag_counts[tag] + len(words))
          for word in words}
    for tag in tags
}

#### Initial Probabilities
  
<ul>
  <li> Compute <i>P</i>(<i>tag</i><sub>start</sub>)): 
      the probability of a tag starting the sentence.
  </li>
  <li> Formula:
      <i>P</i>(<i>tag</i><sub>start</sub>) =
        <span >
            Count(<i>tag</i><sub>start</sub>)
        </span>/
        Total sentences
  </li>
</ul>

In [ ]:
# Compute initial probabilities P(tag | start)
start_counts = defaultdict(int)
total_sentences = len(df['parsed_sentence'])

for sentence in df['parsed_sentence']:
    if sentence:  # Ensure the sentence is not empty
        first_tag = sentence[0][1]
        start_counts[first_tag] += 1

start_probs = {
    tag: (start_counts[tag] + 1) / (total_sentences + len(tags))
    for tag in tags
}

print("\nInitial Probabilities:\n", start_probs)

#### Viterbi Algorithm

<h style="font-size:14px">Now we are using Viterbi Algorithm to find the most likely sequence of hidden states in an HMM.</h>
<p style="font-size:14px">
    <i>V<sub>t</sub></i>(<i>tag</i>) =
        max<sub>prev_tag</sub> [ <i>V<sub>t-1</sub></i>(<i>prev_tag</i>) . P(tag | prev_tag) . P(word | tag) ]
</p>

In [ ]:
lexicon = {
    'the': 'DET',
    'a': 'DET',
    'an': 'DET',
    'this': 'DET',
    'i': 'PRON',
    'you': 'PRON',
    'they': 'PRON',
    'We': 'PRON',
    'he': 'PRON',
    'she': 'PRON',
    'it': 'PRON',
    'is': 'AUX',
    'was': 'AUX',
    'has': 'AUX',
    'but': 'CONJ',
    'and': 'CONJ',  
}

In [ ]:
def viterbi_algorithm(sentence, tags, transition_probs, emission_probs, start_probs):
    V = [{}]  # Viterbi matrix (list of dictionaries)
    backpointer = [{}]  # Backpointer matrix

    # **Lexicon-based heuristic for the first word**
    first_word = sentence[0].lower()
    first_tag = lexicon.get(first_word, None)  # Check if the word exists in the lexicon
    
    for tag in tags:
        if first_tag == tag:  # Give high probability if lexicon match
            V[0][tag] = 1.0
        else:
            V[0][tag] = start_probs.get(tag, 1e-6) * emission_probs[tag].get(first_word, 1e-6)
        
        backpointer[0][tag] = None
    
    # # Initialize the first column of the Viterbi matrix
    # for tag in tags:
    #     V[0][tag] = start_probs.get(tag, 1e-6) * emission_probs[tag].get(sentence[0], 1e-6)
    #     backpointer[0][tag] = None
    
    # Populate the Viterbi matrix
    for t in range(1, len(sentence)):
        V.append({})
        backpointer.append({})
        
        for tag in tags:
            max_prob, best_prev_tag = max(
                (V[t-1][prev_tag] * transition_probs[prev_tag].get(tag, 1e-6) * emission_probs[tag].get(sentence[t], 1e-6), prev_tag)
                for prev_tag in tags
            )
            V[t][tag] = max_prob
            backpointer[t][tag] = best_prev_tag
    
    # Backtrack to find the best sequence
    best_last_tag = max(V[-1], key=V[-1].get)
    best_tags = [best_last_tag]
    
    for t in range(len(sentence) - 1, 0, -1):
        best_tags.insert(0, backpointer[t][best_tags[0]])
    
    return best_tags

<p>Testing on sample sentence to check whether our algorithm working properly or not</p>

In [ ]:
# Sample sentence for demonstration
sample_sentence = ["This", "is", "a", "sample", "sentence", "."]

# Predict tags for the sample sentence using the Viterbi algorithm
predicted_tags_sample = viterbi_algorithm(sample_sentence, tags, transition_probs, emission_probs, start_probs)

# Print the sample sentence with predicted tags
print("Sample Sentence:", sample_sentence)
print("Predicted Tags:", predicted_tags_sample)

#### Evaluation of the HMM

<p>Applying the trained model on Testing dataset and evaluating the Model by giving the accuracy score and the confusion matrix</p>

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Function to predict tags for a given sentence using the Viterbi algorithm
def predict_tags(test_sentence, tags, transition_probs, emission_probs, start_probs):
    return viterbi_algorithm(test_sentence, tags, transition_probs, emission_probs, start_probs)

# Prepare the test data
test_sentences = df_test['parsed_sentence'].tolist()
true_tags = [[tag for _, tag in sentence] for sentence in test_sentences]

# Predict tags for each sentence in the test set
predicted_tags = [predict_tags([word for word, _ in sentence], tags, transition_probs, emission_probs, start_probs) if sentence else [] for sentence in test_sentences]

# Flatten the lists of true and predicted tags
true_tags_flat = [tag for sentence in true_tags for tag in sentence]
predicted_tags_flat = [tag for sentence in predicted_tags for tag in sentence]

# Calculate accuracy
accuracy = accuracy_score(true_tags_flat, predicted_tags_flat)
print(f"Model Accuracy: {accuracy:.3f}")
# Create a DataFrame for incorrectly predicted tags
incorrect_predictions = pd.DataFrame({
    'Word': [word for sentence in test_sentences for word, _ in sentence],
    'True_Tag': true_tags_flat,
    'Predicted_Tag': predicted_tags_flat
})

# Filter out the incorrect predictions
incorrect_predictions = incorrect_predictions[incorrect_predictions['True_Tag'] != incorrect_predictions['Predicted_Tag']]

# Display the first 10 incorrect predictions
print("\nFirst 10 Incorrect Predictions:")
print(incorrect_predictions.head(10))

# Display statistics of incorrect predictions
print("\nStatistics of Incorrect Predictions:")
print(incorrect_predictions.describe(include='all'))

# Generate confusion matrix
conf_matrix = confusion_matrix(true_tags_flat, predicted_tags_flat, labels=tags)
print("\nConfusion Matrix:")
print(conf_matrix)

# Optionally, generate a classification report
class_report = classification_report(true_tags_flat, predicted_tags_flat, labels=tags)
print("\nClassification Report:")
print(class_report)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plotting the confusion matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g', xticklabels=tags, yticklabels=tags)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Tags')
plt.ylabel('True Tags')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()
